In [1]:
import pandas as pd
import numpy as np
import keras
from keras.utils.data_utils import get_file
from keras.preprocessing.image import array_to_img, img_to_array, load_img
from os.path import join
import multiprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

%matplotlib inline

Using TensorFlow backend.


In [2]:
DATA_HOME = '../Facescrub-Faceonly-Cropped/'

In [3]:
dataset = pd.read_csv('facescrub-faceonly.train.csv', nrows=None)

In [4]:
encoder = LabelEncoder()
encoder.fit(dataset['person'])
dataset['person_id'] = encoder.transform(dataset['person'])

In [5]:
dataset.head()

,person,count,path,person_id
0,Hugo Weaving,61,Hugo Weaving/Hugo Weaving_face_65.jpg,165
1,Hugo Weaving,61,Hugo Weaving/Hugo Weaving_face_67.jpg,165
2,Hugo Weaving,61,Hugo Weaving/Hugo Weaving_face_58.jpg,165
3,Hugo Weaving,61,Hugo Weaving/Hugo Weaving_face_61.jpg,165
4,Hugo Weaving,61,Hugo Weaving/Hugo Weaving_face_94.jpg,165


In [6]:
np.mean(dataset['count'])

138.2418320323521

In [6]:
y = dataset['person_id'].as_matrix()

In [7]:
img_paths = [r.path for r in dataset.itertuples()]
print(len(img_paths))

53165


In [8]:
def path2ImgVec(path):
    x = img_to_array(load_img(join(DATA_HOME, path)))
    return x.reshape((1,) + x.shape)

In [9]:
pool = multiprocessing.Pool(8)
results = pool.map(path2ImgVec, img_paths)
pool.close()
pool.join()

In [10]:
X = np.vstack(results)

In [11]:
X.shape

(53165, 55, 47, 3)

In [12]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.1, random_state=42)

In [13]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Dense, Dropout, Activation, Flatten, Input, concatenate
from keras.utils import np_utils
from keras.layers import LSTM
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.metrics import accuracy_score
from keras import metrics
from keras.callbacks import Callback
from keras.layers.normalization import BatchNormalization
from keras.engine import Model
from keras import optimizers

nb_class = len(np.unique(y))
print('nb_class', nb_class)
hidden_dim = 160
best_weights_filepath = '../models/best_weights.facescrub-faceonly.hdf5'

def build_model():
    image_input = Input(shape=X.shape[1:])
    
    conv1 = Conv2D(20, (4, 4), name='conv1')(image_input)
    conv1 = BatchNormalization()(conv1)
    conv1 = Activation('relu')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2), name='pool1')(conv1)
#     pool1 = Dropout(rate=0.2)(pool1)
    
    conv2 = Conv2D(40, (3, 3), name='conv2')(pool1)
    conv2 = BatchNormalization()(conv2)
    conv2 = Activation('relu')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2), name='pool2')(conv2)
#     pool2 = Dropout(rate=0.2)(pool2)

    conv3 = Conv2D(60, (3, 3), name='conv3')(pool2)
    conv3 = BatchNormalization()(conv3)
    conv3 = Activation('relu')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2), name='pool3')(conv3)

    flat1 = Flatten(name='flat1')(pool3)
    
    conv4 = Conv2D(80, (2, 2), name='conv4')(pool3)
    conv4 = BatchNormalization()(conv4)
    conv4 = Activation('relu')(conv4)
    flat2 = Flatten(name='flat2')(conv4)
    
    merged = concatenate([flat1, flat2])
    
    out = Dense(hidden_dim, name='hidden1')(merged)
    out = BatchNormalization()(out)
    out = Activation('relu',name='deepid')(out)
    out = Dense(nb_class, activation='softmax', name='softmax_class')(out)
    
    model = Model(inputs=image_input, outputs=out)

    optimizer = optimizers.Adam(lr=1e-3, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=(1e-4))

    model.compile(
        optimizer=optimizer, #rmsprop
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy'],
    )
    
    print(model.summary())
    return model

nb_class 405


In [19]:
model = build_model()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_2 (InputLayer)             (None, 55, 47, 3)     0                                            
____________________________________________________________________________________________________
conv1 (Conv2D)                   (None, 52, 44, 20)    980                                          
____________________________________________________________________________________________________
batch_normalization_6 (BatchNorm (None, 52, 44, 20)    80                                           
____________________________________________________________________________________________________
activation_5 (Activation)        (None, 52, 44, 20)    0                                            
___________________________________________________________________________________________

In [20]:
saveBestModel = keras.callbacks.ModelCheckpoint(
    best_weights_filepath, 
    monitor='val_acc', 
    verbose=0, 
    save_best_only=True, 
    mode='auto'
)
earlyStopping=keras.callbacks.EarlyStopping(
    monitor='val_loss', 
    patience=10, 
    verbose=1, 
    mode='auto'
)

model.fit(
    X_train, 
    y_train, 
    batch_size=512, 
    epochs=12,
    verbose=2, 
    validation_split=0.1, 
    shuffle=True,
    callbacks=[saveBestModel, earlyStopping],
)

Train on 43063 samples, validate on 4785 samples
Epoch 1/12
5s - loss: 5.4519 - acc: 0.0617 - val_loss: 5.5019 - val_acc: 0.0456
Epoch 2/12
5s - loss: 4.0591 - acc: 0.2848 - val_loss: 4.2499 - val_acc: 0.2510
Epoch 3/12
5s - loss: 2.8766 - acc: 0.5168 - val_loss: 3.5795 - val_acc: 0.3087
Epoch 4/12
5s - loss: 2.0177 - acc: 0.6668 - val_loss: 2.6770 - val_acc: 0.4759
Epoch 5/12
5s - loss: 1.4529 - acc: 0.7586 - val_loss: 2.1336 - val_acc: 0.5858
Epoch 6/12
5s - loss: 1.0718 - acc: 0.8239 - val_loss: 1.8842 - val_acc: 0.6180
Epoch 7/12
5s - loss: 0.8077 - acc: 0.8715 - val_loss: 1.8802 - val_acc: 0.6157
Epoch 8/12
5s - loss: 0.6064 - acc: 0.9116 - val_loss: 1.7498 - val_acc: 0.6330
Epoch 9/12
5s - loss: 0.4530 - acc: 0.9405 - val_loss: 1.8415 - val_acc: 0.6090
Epoch 10/12
5s - loss: 0.3379 - acc: 0.9623 - val_loss: 1.6428 - val_acc: 0.6537
Epoch 11/12
5s - loss: 0.2482 - acc: 0.9776 - val_loss: 1.5706 - val_acc: 0.6748
Epoch 12/12
5s - loss: 0.1765 - acc: 0.9869 - val_loss: 1.5947 - val_

In [24]:
model.load_weights(best_weights_filepath)

In [25]:
model.save('../models/facescrub-faceonly-simple-cnn.model.h5')

In [26]:
model.evaluate(X_test, y_test, batch_size=512, verbose=2)

[1.5270881635792524, 0.68610118571945611]

In [23]:
import gc
gc.collect()

0